In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# ⚙️ 06 – Governance Pipeline: Risk & Compliance for ML\n",
    "\n",
    "This notebook outlines a simplified, modular AI governance process for credit risk ML models.\n",
    "It includes model metadata logging, risk classification, bias detection, explainability and basic audit traceability.\n",
    "\n",
    "These principles support regulatory compliance and internal accountability (e.g. under EU AI Act, BaFin, or ISO 42001)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Imports\n",
    "import pandas as pd\n",
    "from src.data_loader import load_and_preprocess_data\n",
    "from src.model_trainer import train_model\n",
    "from src.risk_assessment import classify_model_risk, detect_bias\n",
    "from src.governance import log_model_metadata, generate_audit_entry"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load and train\n",
    "X_train, X_test, y_train, y_test = load_and_preprocess_data()\n",
    "model = train_model(X_train, y_train)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 1. Log Model Metadata (Traceability)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Log key metadata for governance report\n",
    "log_model_metadata(model_name=\"RandomForestClassifier\",\n",
    "                   version=\"1.0\",\n",
    "                   data_source=\"German Credit Risk\",\n",
    "                   owner=\"ML Security Analyst\",\n",
    "                   purpose=\"Credit risk scoring\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 2. Classify Model Risk (e.g. per EU AI Act)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "risk_level = classify_model_risk(purpose=\"credit_scoring\", sensitive_features=[\"Sex\", \"Age\"])\n",
    "print(\"Risk classification:\", risk_level)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3. Fairness & Bias Detection (e.g. demographic parity)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "bias_report = detect_bias(X_test, y_test, sensitive_feature=\"Sex\", model=model)\n",
    "bias_report"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4. Create audit entry\n",
    "Generate an auditable JSON log that can be stored in GRC tools."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "generate_audit_entry(model_name=\"RandomForestClassifier\", risk_level=risk_level, bias=bias_report)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "- Logged metadata for traceability\n",
    "- Classified model risk based on usage & sensitivity\n",
    "- Detected potential bias based on demographic group\n",
    "- Generated audit entry for review/compliance\n",
    "\n",
    "This pipeline demonstrates a governance layer on top of any AI system – especially valuable in finance and regulated industries."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}